# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from nlppen.extraccion.utils.extraerFechaRecibido import ExtraerFecha
from pyspark.sql.functions import length
from nlppen.extraccion.utils.misc import splitResolucion

import pandas as pd


## Configuracion Spark

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "6")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "12")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/12/09 17:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:
ejecutar = False
if ejecutar:
    terminos = {
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b', r'\bt[éermino (improrrogable)? (de)?\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset/partition', datasets_path='./datasets/estructurales')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
    print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
    estructurales = SentenciasEstructurales(seleccion)

# Procesamiento de todas las variables de sentencias estructurales

## La siguiente celda ejecuta el cálculo de todas las variables definidas para las sentencias estructurales

Nota: Modificar la bandera "ejecutar" para aplicar todo el procesamiento.

In [4]:
from pyspark.sql.types import *

if ejecutar:
    columnas = {
        'se ordena PER' : ArrayType(StringType()),
        'se ordena LOC' : ArrayType(StringType()),
        'se ordena ORG' : ArrayType(StringType()),
        'se ordena MISC' : ArrayType(StringType()),
        'se ordena GPE' : ArrayType(StringType()),
        'se ordena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeOrdena(columnas, True, True)

    columnas = {
        'se condena PER' : ArrayType(StringType()),
        'se condena LOC' : ArrayType(StringType()),
        'se condena ORG' : ArrayType(StringType()),
        'se condena MISC' : ArrayType(StringType()),
        'se condena GPE' : ArrayType(StringType()),
        'se condena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeCondena(columnas, True, True)

    columnas = {
        'extension sentencia' : IntegerType(),
        'extension por lo tanto' : IntegerType()
    }
    estructurales.extraerExtension(columnas, True)

    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType()) #Correción día 27-11-2021. Correr de nuevo
    }
    estructurales.plazosDefinidos(columnas, True)

    columnas = {
        'FechaSolicitud' : TimestampType(),
    }

    estructurales.extrarFechaRecibido(columnas, True)

    columnas = {
        'num resolucion' : StringType()
    }

    estructurales.extraerNumeroSentencia(columnas, True)

    columnas = {
        'inst internacionales' : ArrayType(StringType())
    }

    estructurales.extraerInstrumentosInternacionales(columnas, True)

    columnas = {
        'derechos Norm' : ArrayType(StringType()),
        'derechos GenXPat' : ArrayType(StringType())
    }
    estructurales.extraerDerechos(columnas, True)

    columnas = {
        'derechos Acotados' : ArrayType(StringType()),
        'derechos General' : ArrayType(StringType()),
        'derechos Fundamental' : ArrayType(StringType()),
        'derechos Humano' : ArrayType(StringType())
    }

    estructurales.extraerDerechosSinNormalizar(columnas, True)
    
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
    estructurales.agregarIDSentencia(columnas, sentencias, True)
    
    columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()), #Dicc de Numero de voto y Fecha
        'citasIDVoto' : MapType(StringType(), StringType()) #Dicc de id_Sentencia y # de voto
    }
    estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)
    
    columnas = {
        'seguimientoExt' : StringType(), #Texto extraido de seguimiento
        'seguimientoValue' : IntegerType() #Cantidad de ocurrencias
    }
    estructurales.extraerSeguimiento(columnas, True)
    
    columnas = {
        'subtemaConsiderando' :  ArrayType(StringType())
    }
    estructurales.extraerSubtemaConsiderando(columnas, True)
    #estructurales.seleccion.sdf.toPandas().head(5)
    #Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
    #estructurales.seleccion.guardarDatos()

In [5]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

In [6]:
#df = spark.read.parquet("./dataset/procesado")
#df.toPandas().head()

## Guardar los resultados del preprocesamiento por año

In [7]:
#if ejecutar:
#    df = spark.read.parquet(seleccion.datasets_path+"/terminos.parquet")
#    writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')


## Cargar los resultados preprocesados y divididos por año

In [85]:
if not ejecutar:
    seleccion = Seleccion({}, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesx')
    estructurales = SentenciasEstructurales(seleccion)

In [86]:
#estructurales.seleccion.sdf.select("id_sentencia", "expediente_oficial").toPandas().head(5)

## Ejecutar un criterio

In [87]:
from pyspark.sql.types import *

if not ejecutar:
    columnas = {
        'subtemaConsiderando' :  ArrayType(StringType())
    }
    estructurales.extraerSubtemaConsiderando(columnas, True, False)
    s = estructurales.seleccion.sdf
    s.toPandas().head(5)
    

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [88]:
#s.filter(s.subtemaConsiderando.isNotNull()).toPandas().head(5)

In [89]:
s.filter(s.subtemaConsiderando.isNotNull()).count()

1135

In [90]:
s.filter(s.subtemaConsiderando.isNull()).count()

230

In [91]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
df2 = (s
 .where(s.subtemaConsiderando.isNotNull())
 .select(explode('subtemaConsiderando').alias('subtemaConsiderando'), 'anno')
 .groupby('subtemaConsiderando')
 .count()
 .sort(F.desc('count'))
)

df2.toPandas()

,subtemaConsiderando,count
0,HECHOS PROBADOS,921
1,OBJETO DEL RECURSO,658
2,SOBRE EL FONDO,551
3,HECHOS NO PROBADOS,214
4,CASO CONCRETO,184
5,SOBRE EL CASO CONCRETO,176
6,DOCUMENTACION APORTADA AL EXPEDIENTE,144
7,CONCLUSION,120
8,DOCUMENTACION APORTADA AL EXPEDIENTE,114
9,EL AMPARO SE HA PRESENTADO POR VIOLACION DEL DERECHO A UN PROCEDIMIENTO ADMI...,55


In [77]:
df2 = (s
 .where(s.subtemaConsiderando.isNotNull())
 .select(explode('subtemaConsiderando').alias('subtemaConsiderando'), 'id_sentencia')
)

df2.filter(df2.subtemaConsiderando == "VI").toPandas()

,subtemaConsiderando,id_sentencia


In [66]:
df2 = (s
 .where(s.subtemaConsiderando.isNotNull())
 .select(explode('subtemaConsiderando').alias('subtemaConsiderando'), 'id_sentencia')
)

df2.filter(df2.id_sentencia == "sen-1-0007-742883").toPandas()

,subtemaConsiderando,id_sentencia
0,OBJETO DEL RECURSO,sen-1-0007-742883
1,TAMBIÉN RECLAMA LA DECISIÓN DE LA ARESEP DE AUTORIZAR EL COBRO PARA QUE SU ...,sen-1-0007-742883
2,HECHOS PROBADOS,sen-1-0007-742883
3,C,sen-1-0007-742883
4,SOBRE EL DERECHO DE ACCESO A LA INFORMACIÓN ADMINISTRATIVA,sen-1-0007-742883
5,SOBRE LA INFORMACIÓN SOLICITADA AL INSTITUTO COSTARRICENSE DE ELECTRICIDAD,sen-1-0007-742883
6,C,sen-1-0007-742883
7,C,sen-1-0007-742883
8,SOBRE EL COBRO OBJETADO POR EL RECURRENTE,sen-1-0007-742883
9,C,sen-1-0007-742883


In [17]:
s = estructurales.seleccion.sdf
print(s.filter(s.plazo > 0).count())
print(s.filter(s.plazosDefinidos.isNotNull()).count())
s.filter(s.plazosDefinidos.isNotNull()).filter(s.plazo == 0).select( "id_sentencia", "plazosDefinidos" , "plazo").toPandas()

709
631


/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,id_sentencia,plazosDefinidos,plazo


# Cargar procesamiento de sentencias estructurales.

# Zona de trabajo y cosas utiles

### Sobreescribir el dataset

In [18]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

### Sobreescribir terminos

In [19]:
"""terminos = {
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b', r'\bt[éermino (improrrogable)? (de)?\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesx')
    #seleccion = Seleccion(terminos, spark, parquet_path='./dataset/partition', datasets_path='./datasets/estructurales')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, precargar=False, keepRowEmpty=True, overwriteColumns=True)
    estructurales.seleccion = seleccion"""

'terminos = {\n        \'se ordena\': [r\'\x08se ordena\x08\', r\'\x08se le ordena\x08\', r\'\x08se les ordena\x08\'],\n        \'se condena\': [r\'\x08se condena\x08\', r\'\x08se le condena\x08\', r\'\x08se les condena\x08\'],\n        \'plan\': [r\'\x08plan\x08\'],\n        \'plazo\': [r\'\x08plazo\x08\', r\'\x08t[éermino (improrrogable)? (de)?\x08\']\n    }\n    seleccion = Seleccion(terminos, spark, parquet_path=\'./dataset/procesado\', datasets_path=\'./datasets/estructuralesx\')\n    #seleccion = Seleccion(terminos, spark, parquet_path=\'./dataset/partition\', datasets_path=\'./datasets/estructurales\')\n    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))\n    seleccion.filtrar_sentencias(preprocess=solo_portanto, precargar=False, keepRowEmpty=True, overwriteColumns=True)\n    estructurales.seleccion = seleccion'

### Funciones pandas y visualizacion de datos

In [20]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)



### Funcion de testing

In [106]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.extraccion.utils.Txt2Date import Txt2Date
import re

def testing(txt):
    print(txt)
    regularExpresion = re.compile(r"(?<=[LXVI])[\.|-]+\s[A-Z|\s|ÁÉÍÓÚ]+(?=[:\.])")
    extraccion = regularExpresion.findall(txt.upper())
    for match in extraccion:
        print("-*-*-*-*-*")
        print("Match", match)
        match = re.sub("\.|\n", "", match)
        matchNumber = re.match("[LXVI]+", match)
        print(matchNumber)
        print("Nuevo", match)
    

### Recorrer dataset con una funcion de testing

In [112]:

from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy

#citas = s.filter(df2.id_sentencia == "sen-1-0007-742883").toPandas()
citas = s.filter(s.subtemaConsiderando.isNull()).toPandas()
#citas = s.filter(s.anno == "2015").toPandas()
#citas = s.filter(s.CitaSentencias_ents.isNotNull()).toPandas()
#nlp = spark_get_spacy("es_core_news_lg")
#size = len(citas)
size = 1
#print(citas.columns)
for i in range(2, 3):
    sent = citas.iloc[i, 4]
    id_sentencia = citas.iloc[i, -9]
    txt = citas.iloc[i, 1]
    print("******************************")
    print(sent)
    print(id_sentencia)
    termino_ext = citas.iloc[i, 8]
    testing(solo_considerando(txt))

******************************
090074210007CO
sen-1-0007-455274



### Cargar spacy

In [24]:
"""nlp = spark_get_spacy('es_core_news_lg')
doc = nlp("plan remedial")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
"""

'nlp = spark_get_spacy(\'es_core_news_lg\')\ndoc = nlp("plan remedial")\nfor token in doc:\n    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)\n'